In [90]:
import json
from coinbase.rest import RESTClient
from json import dumps
import pandas as pd
from random import randint
import pytz
import datetime
import time

In [51]:
# Get demo API key
def get_demo_key(key_type):
    f = open(f"/home/vikas/Documents/Demo_{key_type}_CG_key.json")
    key_dict = json.load(f)
    return key_dict["name"], key_dict["privateKey"]

In [55]:
api_key, api_secret = get_demo_key("view")

view_client = RESTClient(api_key = api_key,
                         api_secret = api_secret)

## Test API

In [53]:
server_time = view_client.get_unix_time()
print(dumps(server_time, indent = 4))

{
    "iso": "2024-09-02T19:36:08Z",
    "epochSeconds": "1725305768",
    "epochMillis": "1725305768873"
}


In [54]:
view_client.get_api_key_permissions()

{'can_view': True,
 'can_trade': False,
 'can_transfer': False,
 'portfolio_uuid': 'd083ea15-3f44-54da-8e9e-2a5a9bdccb32'}

## Convert to local timezone

In [101]:
def convert_to_local_tz(old_ts):
    
    new_tz = pytz.timezone("Europe/Amsterdam")
    old_tz = pytz.timezone("UTC")
    
    format = "%Y-%m-%dT%H:%M:%S.%fZ"
    datetime_obj = datetime.datetime.strptime(old_ts, format)
    
    localized_ts = old_tz.localize(datetime_obj)
    new_ts = localized_ts.astimezone(new_tz)
    
    return new_ts

## Get trades for a given pair

In [58]:
# btc_usd = view_client.get_public_market_trades("BTC-USD", 2)
# print(dumps(btc_usd, indent = 4))

In [104]:
def get_latest_trades(pair, num_trades):
    
    all_trades = view_client.get_public_market_trades(pair, num_trades)
    
    df_trades = pd.DataFrame(all_trades["trades"])
    df_trades["time"] = df_trades["time"].apply(convert_to_local_tz)
    
    return df_trades

In [105]:
df_btc_usd = get_latest_trades("ETH-EUR", 5)
df_btc_usd

,trade_id,product_id,price,size,time,side,bid,ask
0,53256926,ETH-EUR,2294.2,0.00280435,2024-09-02 22:36:31.069523+02:00,SELL,,
1,53256925,ETH-EUR,2294.55,0.04189089,2024-09-02 22:36:25.338282+02:00,SELL,,
2,53256924,ETH-EUR,2295.73,0.03813667,2024-09-02 22:36:12.482725+02:00,SELL,,
3,53256923,ETH-EUR,2295.68,0.13130917,2024-09-02 22:36:12.482725+02:00,SELL,,
4,53256922,ETH-EUR,2297.37,1.73677525,2024-09-02 22:36:08.081227+02:00,BUY,,


## Get accounts

In [46]:
#client.get_accounts()

## Get portfolios

In [50]:
client.get_portfolios()

{'portfolios': [{'name': 'Default',
   'uuid': 'd083ea15-3f44-54da-8e9e-2a5a9bdccb32',
   'type': 'DEFAULT',
   'deleted': False},
  {'name': 'Demo portfolio',
   'uuid': '581b97fc-d5a9-4b69-bb3c-1c9853fb9b71',
   'type': 'CONSUMER',
   'deleted': False}]}

## Check portfolio breakdown

In [60]:
api_key, api_secret = get_demo_key("trade")

trade_client = RESTClient(api_key = api_key,
                          api_secret = api_secret)

In [61]:
trade_client.get_portfolio_breakdown('581b97fc-d5a9-4b69-bb3c-1c9853fb9b71')

{'breakdown': {'portfolio': {'name': 'Demo portfolio',
   'uuid': '581b97fc-d5a9-4b69-bb3c-1c9853fb9b71',
   'type': 'CONSUMER',
   'deleted': False},
  'portfolio_balances': {'total_balance': {'value': '20', 'currency': 'EUR'},
   'total_futures_balance': {'value': '0', 'currency': 'EUR'},
   'total_cash_equivalent_balance': {'value': '20', 'currency': 'EUR'},
   'total_crypto_balance': {'value': '0', 'currency': 'EUR'},
   'futures_unrealized_pnl': {'value': '0', 'currency': 'EUR'},
   'perp_unrealized_pnl': {'value': '0', 'currency': 'EUR'}},
  'spot_positions': [{'asset': 'EUR',
    'account_uuid': '250a2cb1-2aa3-5b0b-bb56-a17cbc2f9780',
    'total_balance_fiat': 20,
    'total_balance_crypto': 20,
    'available_to_trade_fiat': 20,
    'allocation': 1,
    'cost_basis': {'value': '20', 'currency': 'EUR'},
    'asset_img_url': '',
    'is_cash': True,
    'average_entry_price': {'value': '1.00', 'currency': 'EUR'},
    'asset_uuid': '',
    'available_to_trade_crypto': 20,
    'unr

## Place a market order

In [68]:
def generate_client_id(id_length):
    
    return "".join(f"{randint(0, 9)}" for _ in range(id_length))       

In [106]:
# buy_order = trade_client.market_order_buy(
#     client_order_id = generate_client_id(10),
#     product_id = "ETH-EUR",
#     quote_size = "5")   

In [107]:
buy_order

{'success': True,
 'failure_reason': 'UNKNOWN_FAILURE_REASON',
 'order_id': '325b80c0-3742-4fc2-96ba-27f66d617840',
 'success_response': {'order_id': '325b80c0-3742-4fc2-96ba-27f66d617840',
  'product_id': 'ETH-EUR',
  'side': 'BUY',
  'client_order_id': '2290779562',
  'attached_order_id': ''},
 'order_configuration': {'market_market_ioc': {'quote_size': '5'}}}

In [81]:
# fills = trade_client.get_fills(order_id = buy_order["order_id"])
# fills

In [83]:
#trade_client.get_order('325b80c0-3742-4fc2-96ba-27f66d617840')